In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import tensorflow as tf
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from PIL.Image import BILINEAR
from multiprocessing import Process, freeze_support, set_start_method

import argparse
import numpy as np
import time
import uuid
import os
import sys
from importlib import reload
from models.resnet import *

import augment
import util
from logger import Logger
from custom_dataset import MultiViewDataSet
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
try:
    !~/copydata2local.sh MESAandSPIROMICS_Views
except:
    pass
        

input is MESAandSPIROMICS_Views
SPIROMICS = 2630
MESA = 3163
Combined = 5793
(5793,)
(5572,)
5572
279380
txtImages already exists
sets already exists
Subject JH111067 has only 0 views!
Subject JH113975 has only 0 views!
Subject JH114172 has only 0 views!
Subject JH114311 has only 0 views!
Subject LA192113 has only 0 views!
Subject LA192611 has only 0 views!
Subject LA193698 has only 0 views!
Subject MU160548 has only 0 views!
Subject MU161027 has only 0 views!
Subject MU161354 has only 0 views!
Subject MU161836 has only 0 views!
Subject MU163724 has only 0 views!
Subject SF180987 has only 0 views!
Subject SF182815 has only 0 views!
Subject SF182842 has only 0 views!
Subject SF183565 has only 0 views!
Subject UA240150 has only 0 views!
Subject UA240503 has only 0 views!
Subject UA240902 has only 0 views!
Subject UI250228 has only 0 views!
Subject UI250537 has only 0 views!
Subject UI250697 has only 0 views!
Subject UT171852 has only 0 views!
Subject UT172380 has only 0 views!
Subject UT

In [6]:
parser = argparse.ArgumentParser(description='MVCNN-PyTorch')
parser.add_argument('--data', metavar='DIR', default='/localscratch/Users/amotahari/MESAandSPIROMICS_Views', help='path to dataset')
parser.add_argument('-j', '--job_id', metavar='ID', default='', help='SGE job ID')
parser.add_argument('--resnet', default=18, choices=[18, 34, 50, 101, 152], type=int, metavar='N', help='resnet depth (default: resnet18)')
parser.add_argument('--epochs', default=10000, type=int, metavar='N', help='number of total epochs to run (default: 100)')
parser.add_argument('-b', '--batch-size', default=48, type=int, metavar='N', help='mini-batch size (default: 4)')
parser.add_argument('--lr', '--learning-rate', default=0.0001, type=float, metavar='LR', help='initial learning rate (default: 0.01)')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M', help='momentum (default: 0.9)')
parser.add_argument('--lr-decay-freq', default=200, type=float, metavar='W', help='learning rate decay (default: 30)')
parser.add_argument('--lr-decay', default=1.0, type=float, metavar='W', help='learning rate decay (default: 0.1)')
parser.add_argument('--print-freq', '-p', default=10, type=int, metavar='N', help='print frequency (default: 10)')
parser.add_argument('-r', '--resume', default='/Shared/CTmechanics_COPDGene/Amin/Airway_PyTorch/607310/checkpoint.pth.tar', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
parser.add_argument('-o', '--output', default='/Shared/CTmechanics_COPDGene/Amin/Airway_PyTorch', type=str, metavar='PATH',
                    help='path to Output folder for logs and checkpoints (default: none)')
parser.add_argument('-w', '--workers', default=0, type=int, metavar='N', help='Number of workers in input pipe (default: 4)')
parser.add_argument('-wd', '--weight_decay', default=0.0, type=float, metavar='W', help='Weight decay factor (default: 0.1)')
parser.add_argument('--l1weight', default=0.0, type=float, metavar='W', help='L1 Regularization Weight (default: 0.0)')
parser.add_argument('--mode', default='train', type=str, metavar='M', help='Operating mode (default: train)')
parser.add_argument('--view_step', default=2, type=int, metavar='N', help='Steps in selecting views (default: 1)')



parser.add_argument('-f', '--fun', default='', type=str, metavar='PATH',
                    help='path to Output folder for logs and checkpoints (default: none)')
#parser.add_argument('--pretrained', dest='pretrained', action='store_true', help='use pre-trained model')

args = parser.parse_args()

#args.mode = 'test'
args.resume = ''
#args.l1weight = 1.0

print(args)

Namespace(batch_size=48, data='/localscratch/Users/amotahari/MESAandSPIROMICS_Views', epochs=10000, fun='/Users/amotahari/.local/share/jupyter/runtime/kernel-8ff8ac8e-0f91-43b3-8c8a-525bdae3edc7.json', job_id='', l1weight=0.0, lr=0.0001, lr_decay=1.0, lr_decay_freq=200, mode='train', momentum=0.9, output='/Shared/CTmechanics_COPDGene/Amin/Airway_PyTorch', print_freq=10, resnet=18, resume='', view_step=2, weight_decay=0.0, workers=0)


In [3]:
transform = transforms.Compose([
    #transforms.CenterCrop(500),
    #transforms.RandomAffine(30, translate=(.2,.2), scale=None, shear=None, resample=BILINEAR, fillcolor=0), # Augmentation
    transforms.Resize(224),
    transforms.ToTensor(),
])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('Loading data')
if args.mode=='test':
    dset_test = MultiViewDataSet(args.data, 'test', step = args.view_step, transform=transform)
    nofviews = dset_test.nofviews
    print("\nTest Data Loaded!")
else:
    dset_train = MultiViewDataSet(args.data, 'train', step = args.view_step, transform=transform)
    print("\nTraining Data Loaded!")
    dset_val = MultiViewDataSet(args.data, 'validation', step = args.view_step, transform=transform)
    print("\nValidation Data Loaded!")
    nofviews = dset_train.nofviews
    
print("Number of views per subject =", nofviews)

Loading data
/localscratch/Users/amotahari/MESAandSPIROMICS_Views/sets/train.txt
Loading train data: 100% 
Training Data Loaded!
/localscratch/Users/amotahari/MESAandSPIROMICS_Views/sets/validation.txt
Loading validation data: 100% 
Validation Data Loaded!
Number of views per subject = 24


In [7]:
if args.mode=='test':
    test_loader = DataLoader(dset_test, batch_size=args.batch_size, shuffle=False, num_workers=args.workers)
    classes = dset_test.classes
else:
    val_loader = DataLoader(dset_val, batch_size=args.batch_size, shuffle=True, num_workers=args.workers)
    train_loader = DataLoader(dset_train, batch_size=args.batch_size, shuffle=True, num_workers=args.workers)
    classes = dset_train.classes

In [8]:
#torch.cuda.empty_cache()

if args.job_id =='':
    job_folder = "Jupyter_"+time.asctime( time.localtime(time.time()) )#str(uuid.uuid4().hex)[:5]
    job_folder = job_folder.replace(' ','_').replace(':','_')
else:
    job_folder = args.job_id

print(job_folder)

args.output = os.path.join(args.output, job_folder)
if not os.path.exists(args.output):
    os.makedirs(args.output)

with open(os.path.join(args.output,'Arguments.txt'), "w") as text_file:
    print(args, file = text_file) #text_file.write(args)
    
#classes = dset_train.classes

print(len(classes), classes)

if args.resnet == 18:
    resnet = resnet18(num_classes=len(classes))
elif args.resnet == 34:
    resnet = resnet34(num_classes=len(classes))
elif args.resnet == 50:
    resnet = resnet50(num_classes=len(classes))
elif args.resnet == 101:
    resnet = resnet101(num_classes=len(classes))
elif args.resnet == 152:
    resnet = resnet152(num_classes=len(classes))

print('Using resnet' + str(args.resnet))
resnet.to(device)
device_ids = range(torch.cuda.device_count())
print("CUDA devices available: ",device_ids)
resnet = nn.DataParallel(resnet, device_ids=device_ids)
cudnn.benchmark = True

print('Running on ' + str(device))

logger = Logger(os.path.join(args.output, 'logs'))

# Loss and Optimizer
lr = args.lr
n_epochs = args.epochs
weight=torch.tensor(1/np.array([.63,.30,.06]),dtype=torch.float32,device=device)
print('Class Weights = {}'.format(weight))
criterion = nn.CrossEntropyLoss(weight=weight)
optimizer = torch.optim.Adam(resnet.parameters(), lr=lr, weight_decay=args.weight_decay)
fc_layers = []
_,model = next(resnet.named_children())
for name,layer in model.named_children():
    if name in ['fc','fc0']:
        fc_layers.append(layer)

print('FC Layers (for L1 regularization:)', fc_layers)

best_acc = 0.0
best_loss = 0.0
start_epoch = 0

# Helper functions
def load_checkpoint():
    global best_acc, start_epoch
    # Load checkpoint.
    print('\n==> Loading checkpoint..')
    assert os.path.isfile(args.resume), 'Error: no checkpoint file found!'

    checkpoint = torch.load(args.resume)
    best_acc = checkpoint['best_acc']
    start_epoch = checkpoint['epoch']
    resnet.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])


def train():
    train_size = len(train_loader)
    training_loss = []
    for i, (inputs, targets) in enumerate(train_loader):
        # Convert from list of 3D to 4D
        #inputs = np.stack(inputs, axis=1)
        #inputs = np.stack(inputs, axis=0)
        inputs = augmentor.augment_on_GPU(inputs)

        #print("shape of Train input= ", inputs.shape)        
        #inputs = augment_on_GPU(inputs)
        inputs = torch.from_numpy(inputs)
        #print("shape of Train input from numpy= ", inputs.shape)  
        inputs, targets = inputs.cuda(), targets.cuda(0)
        inputs, targets = Variable(inputs), Variable(targets)
        
        # compute output
        outputs = resnet(inputs)
        #print(outputs.get_device(), targets.get_device())
        #print(fc.weight for fc in fc_layers)
        l1sum = 0.0;
        for fc_layer in fc_layers:
            l1sum += torch.mean(torch.abs(fc_layer.weight))
        regularization_loss = args.l1weight*l1sum
        loss = criterion(outputs, targets)
        loss += regularization_loss
        training_loss.append(loss)
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % args.print_freq == 0:
            #print(regularization_loss)
            print("\tIter [%d/%d] Loss: %.4f, Reg: %.6f" % (i + 1, train_size, loss.item(), regularization_loss.item()))
    return torch.mean(torch.stack(training_loss))

# Validation and Testing
def eval(data_loader, is_test=False):
    if is_test:
        load_checkpoint()

    # Eval
    total = 0.0
    correct = 0.0

    total_loss = 0.0
    n = 0

    for i, (inputs, targets) in enumerate(data_loader):
        with torch.no_grad():
            # Convert from list of 3D to 4D
            #inputs = np.stack(inputs, axis=0)
            inputs = augmentor.augment_on_GPU(inputs)
            
            #print("shape of Val input= ", inputs.shape)
            #inputs = augment_on_GPU(inputs)
            inputs = torch.from_numpy(inputs)

            inputs, targets = inputs.cuda(), targets.cuda(0)
            inputs, targets = Variable(inputs), Variable(targets)

            # compute output
            outputs = resnet(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss
            n += 1

            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted.cpu() == targets.cpu()).sum()

    avg_test_acc = 100 * correct / total
    avg_loss = total_loss / n

    return avg_test_acc, avg_loss

test_output=[]
test_target=[]

def test(data_loader):
    resnet.eval()
    # Eval
    total = 0.0
    correct = 0.0

    total_loss = 0.0
    n = 0

    for i, (inputs, targets) in enumerate(data_loader):
        with torch.no_grad():
            # Convert from list of 3D to 4D
            #inputs = np.stack(inputs, axis=0)
            A = inputs
            inputs = augmentor.augment_on_GPU(inputs)
            #print("shape of Val input= ", inputs.shape)
            #inputs = augment_on_GPU(inputs)
            inputs = torch.from_numpy(inputs)
            #test_target.append(targets.data.numpy())
            #test_target = targets.data.numpy()
            inputs, targets = inputs.cuda(), targets.cuda(0)
            inputs, targets = Variable(inputs), Variable(targets)

            # compute output
            outputs = resnet(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss
            n += 1

            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted.cpu() == targets.cpu()).sum()
            
            test_output.append(outputs.cpu().data.numpy())
            test_target.append(targets.cpu().data.numpy())
            '''
            for V in range(1):
                #if test_target[i] == 1:
                B = A[1,V,:].transpose(0, 2)
                I1 = B.transpose(0,1)
                I1 = (I1.data.numpy())
                I = inputs.cpu()
                B = I[1,V,:].transpose(0, 2)
                I2 = B.transpose(0,1)
                I2 = (I2.data.numpy()) 

                #print(np.amin(I2-I1))
                #print(np.amax(I2-I1))
                fig = plt.figure(figsize=(40, 20))
                plt.subplot(1,2,1)
                plt.imshow(1-I1)
                plt.axis('off')                
                plt.subplot(1,2,2)
                plt.imshow(1-I2)
                plt.axis('off')
                #fig.savefig('/Shared/CTmechanics_COPDGene/Amin/Report/COPDGene/Augmentation/Augmentation_{:02}.png'.format(V))
            '''    
    #print(test_target)
    #print(test_output)
            
            
    avg_test_acc = 100 * correct / total
    avg_loss = total_loss / n

    return avg_test_acc, avg_loss

# Training / Eval loop
reload(augment)
augmentor = augment.augmentor(nofviews,ngpus = len(device_ids))

if args.resume or args.mode == 'test':
    load_checkpoint()

    
if args.mode == 'test':
    print('Testing:')
    avg_test_acc, avg_loss = test(test_loader)
    print('\tVal Acc: %.2f - Loss: %.4f' % (avg_test_acc.item(), avg_loss.item()))
    print('\tCurrent best val acc: %.2f' % best_acc)    
    
else:
    for epoch in range(start_epoch, n_epochs):
        print('\n-----------------------------------')
        print('Epoch: [%d/%d]' % (epoch+1, n_epochs))
        start = time.time()

        resnet.train()
        training_loss = train()
        print('Training loss = %.4f' % training_loss)
        print('Time taken: %.2f sec.' % (time.time() - start))
        if (epoch + 1) % 1 == 0: # Eval every 5 epoch
            resnet.eval()
            avg_test_acc, avg_loss = eval(val_loader)

            print('\nEvaluation:')
            print('\tVal Acc: %.2f - Loss: %.4f' % (avg_test_acc.item(), avg_loss.item()))
            print('\tCurrent best val acc: %.2f' % best_acc)

            # Log epoch to tensorboard
            # See log using: tensorboard --logdir='logs' --port=6006
            util.logEpoch(logger, resnet, epoch + 1, training_loss , avg_loss, avg_test_acc)

            # Save model
            if avg_test_acc > best_acc:
                print('\tSaving checkpoint - Acc: %.2f' % avg_test_acc)
                best_acc = avg_test_acc
                best_loss = avg_loss
                util.save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': resnet.state_dict(),
                    'acc': avg_test_acc,
                    'best_acc': best_acc,
                    'optimizer': optimizer.state_dict(),
                    },
                    checkpoint = args.output
                )

        # Decaying Learning Rate
        if (epoch + 1) % args.lr_decay_freq == 0:
            lr *= args.lr_decay
            optimizer = torch.optim.Adam(resnet.parameters(), lr=lr, weight_decay=args.weight_decay)
            print('Learning rate:', lr)

Jupyter_Tue_Apr_23_23_19_39_2019
3 ['Standard', 'AccB', 'AbsRB7']
Using resnet18
CUDA devices available:  range(0, 4)
Running on cuda:0
Class Weights = tensor([  1.5873,   3.3333,  16.6667], device='cuda:0')
FC Layers (for L1 regularization:) [Linear(in_features=512, out_features=3, bias=True)]
Number of GPUs:  4

-----------------------------------
Epoch: [1/10000]
	Iter [10/68] Loss: 1.2121, Reg: 0.000000
	Iter [20/68] Loss: 1.1593, Reg: 0.000000
	Iter [30/68] Loss: 1.1240, Reg: 0.000000
	Iter [40/68] Loss: 1.0897, Reg: 0.000000
	Iter [50/68] Loss: 1.0398, Reg: 0.000000
	Iter [60/68] Loss: 1.0969, Reg: 0.000000
Training loss = 1.1063
Time taken: 156.52 sec.

Evaluation:
	Val Acc: 6.00 - Loss: 1.1961
	Current best val acc: 0.00
	Saving checkpoint - Acc: 6.00

-----------------------------------
Epoch: [2/10000]
	Iter [10/68] Loss: 1.2663, Reg: 0.000000
	Iter [20/68] Loss: 1.2255, Reg: 0.000000
	Iter [30/68] Loss: 1.0298, Reg: 0.000000
	Iter [40/68] Loss: 1.0264, Reg: 0.000000
	Iter [5

	Iter [30/68] Loss: 1.0776, Reg: 0.000000
	Iter [40/68] Loss: 1.0102, Reg: 0.000000
	Iter [50/68] Loss: 0.9279, Reg: 0.000000
	Iter [60/68] Loss: 1.0009, Reg: 0.000000
Training loss = 0.9987
Time taken: 150.60 sec.

Evaluation:
	Val Acc: 32.00 - Loss: 1.1367
	Current best val acc: 71.00

-----------------------------------
Epoch: [20/10000]
	Iter [10/68] Loss: 1.0030, Reg: 0.000000
	Iter [20/68] Loss: 0.8707, Reg: 0.000000
	Iter [30/68] Loss: 0.9549, Reg: 0.000000
	Iter [40/68] Loss: 1.0375, Reg: 0.000000
	Iter [50/68] Loss: 1.0414, Reg: 0.000000
	Iter [60/68] Loss: 0.8218, Reg: 0.000000
Training loss = 1.0007
Time taken: 149.25 sec.

Evaluation:
	Val Acc: 33.00 - Loss: 1.0905
	Current best val acc: 71.00

-----------------------------------
Epoch: [21/10000]
	Iter [10/68] Loss: 1.1457, Reg: 0.000000
	Iter [20/68] Loss: 0.8500, Reg: 0.000000
	Iter [30/68] Loss: 0.8481, Reg: 0.000000
	Iter [40/68] Loss: 0.8085, Reg: 0.000000
	Iter [50/68] Loss: 1.0539, Reg: 0.000000
	Iter [60/68] Loss: 

	Iter [50/68] Loss: 0.8346, Reg: 0.000000
	Iter [60/68] Loss: 0.8487, Reg: 0.000000
Training loss = 0.8386
Time taken: 133.54 sec.

Evaluation:
	Val Acc: 36.00 - Loss: 1.0846
	Current best val acc: 71.00

-----------------------------------
Epoch: [39/10000]
	Iter [10/68] Loss: 0.9768, Reg: 0.000000
	Iter [20/68] Loss: 0.5866, Reg: 0.000000
	Iter [30/68] Loss: 1.0767, Reg: 0.000000
	Iter [40/68] Loss: 0.5935, Reg: 0.000000
	Iter [50/68] Loss: 0.6652, Reg: 0.000000
	Iter [60/68] Loss: 0.8995, Reg: 0.000000
Training loss = 0.8362
Time taken: 126.98 sec.

Evaluation:
	Val Acc: 48.00 - Loss: 0.8645
	Current best val acc: 71.00

-----------------------------------
Epoch: [40/10000]
	Iter [10/68] Loss: 0.6220, Reg: 0.000000
	Iter [20/68] Loss: 0.6240, Reg: 0.000000
	Iter [30/68] Loss: 0.9887, Reg: 0.000000
	Iter [40/68] Loss: 0.8777, Reg: 0.000000
	Iter [50/68] Loss: 0.8266, Reg: 0.000000
	Iter [60/68] Loss: 0.6476, Reg: 0.000000
Training loss = 0.8084
Time taken: 147.88 sec.

Evaluation:
	V

Training loss = 0.7264
Time taken: 141.92 sec.

Evaluation:
	Val Acc: 56.00 - Loss: 0.8129
	Current best val acc: 71.00

-----------------------------------
Epoch: [58/10000]
	Iter [10/68] Loss: 0.8343, Reg: 0.000000
	Iter [20/68] Loss: 0.5922, Reg: 0.000000
	Iter [30/68] Loss: 0.6378, Reg: 0.000000
	Iter [40/68] Loss: 0.6294, Reg: 0.000000
	Iter [50/68] Loss: 0.5743, Reg: 0.000000
	Iter [60/68] Loss: 0.6011, Reg: 0.000000
Training loss = 0.7307
Time taken: 144.61 sec.

Evaluation:
	Val Acc: 49.00 - Loss: 0.8222
	Current best val acc: 71.00

-----------------------------------
Epoch: [59/10000]
	Iter [10/68] Loss: 0.8382, Reg: 0.000000
	Iter [20/68] Loss: 0.6986, Reg: 0.000000
	Iter [30/68] Loss: 1.0597, Reg: 0.000000
	Iter [40/68] Loss: 0.8972, Reg: 0.000000
	Iter [50/68] Loss: 0.7000, Reg: 0.000000
	Iter [60/68] Loss: 0.6628, Reg: 0.000000
Training loss = 0.7171
Time taken: 149.98 sec.

Evaluation:
	Val Acc: 44.00 - Loss: 0.8374
	Current best val acc: 71.00

-------------------------


-----------------------------------
Epoch: [77/10000]
	Iter [10/68] Loss: 0.6355, Reg: 0.000000
	Iter [20/68] Loss: 0.7663, Reg: 0.000000
	Iter [30/68] Loss: 0.4745, Reg: 0.000000
	Iter [40/68] Loss: 0.6832, Reg: 0.000000
	Iter [50/68] Loss: 0.4693, Reg: 0.000000
	Iter [60/68] Loss: 0.6991, Reg: 0.000000
Training loss = 0.6716
Time taken: 149.94 sec.

Evaluation:
	Val Acc: 41.00 - Loss: 0.8967
	Current best val acc: 71.00

-----------------------------------
Epoch: [78/10000]
	Iter [10/68] Loss: 0.4576, Reg: 0.000000
	Iter [20/68] Loss: 0.7696, Reg: 0.000000
	Iter [30/68] Loss: 0.5772, Reg: 0.000000
	Iter [40/68] Loss: 0.5923, Reg: 0.000000
	Iter [50/68] Loss: 0.6699, Reg: 0.000000
	Iter [60/68] Loss: 0.5131, Reg: 0.000000
Training loss = 0.6415
Time taken: 150.59 sec.

Evaluation:
	Val Acc: 47.00 - Loss: 0.6959
	Current best val acc: 71.00

-----------------------------------
Epoch: [79/10000]
	Iter [10/68] Loss: 0.6160, Reg: 0.000000
	Iter [20/68] Loss: 0.7264, Reg: 0.000000
	Iter [

...


Evaluation:
	Val Acc: 64.00 - Loss: 0.6448
	Current best val acc: 75.00

-----------------------------------
Epoch: [173/10000]
	Iter [10/68] Loss: 0.3404, Reg: 0.000000
	Iter [20/68] Loss: 0.3853, Reg: 0.000000
	Iter [30/68] Loss: 0.3537, Reg: 0.000000
	Iter [40/68] Loss: 0.3722, Reg: 0.000000
	Iter [50/68] Loss: 0.8709, Reg: 0.000000
	Iter [60/68] Loss: 0.3415, Reg: 0.000000
Training loss = 0.4361
Time taken: 141.20 sec.

Evaluation:
	Val Acc: 68.00 - Loss: 0.5701
	Current best val acc: 75.00

-----------------------------------
Epoch: [174/10000]
	Iter [10/68] Loss: 0.5349, Reg: 0.000000
	Iter [20/68] Loss: 0.4567, Reg: 0.000000
	Iter [30/68] Loss: 0.3469, Reg: 0.000000
	Iter [40/68] Loss: 0.4114, Reg: 0.000000
	Iter [50/68] Loss: 0.4888, Reg: 0.000000
	Iter [60/68] Loss: 0.4065, Reg: 0.000000
Training loss = 0.4494
Time taken: 149.45 sec.

Evaluation:
	Val Acc: 65.00 - Loss: 0.6275
	Current best val acc: 75.00

-----------------------------------
Epoch: [175/10000]
	Iter [10/68] L


-----------------------------------
Epoch: [192/10000]
	Iter [10/68] Loss: 0.2684, Reg: 0.000000
	Iter [20/68] Loss: 0.3126, Reg: 0.000000
	Iter [30/68] Loss: 0.4076, Reg: 0.000000
	Iter [40/68] Loss: 0.4838, Reg: 0.000000
	Iter [50/68] Loss: 0.4365, Reg: 0.000000
	Iter [60/68] Loss: 0.4464, Reg: 0.000000
Training loss = 0.3833
Time taken: 144.41 sec.

Evaluation:
	Val Acc: 77.00 - Loss: 0.6052
	Current best val acc: 76.00
	Saving checkpoint - Acc: 77.00

-----------------------------------
Epoch: [193/10000]
	Iter [10/68] Loss: 0.2743, Reg: 0.000000
	Iter [20/68] Loss: 0.4877, Reg: 0.000000
	Iter [30/68] Loss: 0.5386, Reg: 0.000000
	Iter [40/68] Loss: 0.3620, Reg: 0.000000
	Iter [50/68] Loss: 0.4059, Reg: 0.000000
	Iter [60/68] Loss: 0.3483, Reg: 0.000000
Training loss = 0.4255
Time taken: 149.17 sec.

Evaluation:
	Val Acc: 67.00 - Loss: 0.6338
	Current best val acc: 77.00

-----------------------------------
Epoch: [194/10000]
	Iter [10/68] Loss: 0.4383, Reg: 0.000000
	Iter [20/68] 

	Iter [10/68] Loss: 0.2548, Reg: 0.000000
	Iter [20/68] Loss: 0.4435, Reg: 0.000000
	Iter [30/68] Loss: 0.4493, Reg: 0.000000
	Iter [40/68] Loss: 0.2865, Reg: 0.000000
	Iter [50/68] Loss: 0.3254, Reg: 0.000000
	Iter [60/68] Loss: 0.4947, Reg: 0.000000
Training loss = 0.3563
Time taken: 136.77 sec.

Evaluation:
	Val Acc: 71.00 - Loss: 0.6671
	Current best val acc: 77.00

-----------------------------------
Epoch: [212/10000]
	Iter [10/68] Loss: 0.3302, Reg: 0.000000
	Iter [20/68] Loss: 0.3651, Reg: 0.000000
	Iter [30/68] Loss: 0.4006, Reg: 0.000000
	Iter [40/68] Loss: 0.3223, Reg: 0.000000
	Iter [50/68] Loss: 0.3187, Reg: 0.000000
	Iter [60/68] Loss: 0.3397, Reg: 0.000000
Training loss = 0.3747
Time taken: 146.67 sec.

Evaluation:
	Val Acc: 55.00 - Loss: 0.8291
	Current best val acc: 77.00

-----------------------------------
Epoch: [213/10000]
	Iter [10/68] Loss: 0.3393, Reg: 0.000000
	Iter [20/68] Loss: 0.3790, Reg: 0.000000
	Iter [30/68] Loss: 0.3507, Reg: 0.000000
	Iter [40/68] Loss

	Iter [30/68] Loss: 0.4360, Reg: 0.000000
	Iter [40/68] Loss: 0.1800, Reg: 0.000000
	Iter [50/68] Loss: 0.4272, Reg: 0.000000
	Iter [60/68] Loss: 0.3144, Reg: 0.000000
Training loss = 0.3586
Time taken: 144.55 sec.

Evaluation:
	Val Acc: 65.00 - Loss: 0.7047
	Current best val acc: 77.00

-----------------------------------
Epoch: [231/10000]
	Iter [10/68] Loss: 0.2510, Reg: 0.000000
	Iter [20/68] Loss: 0.4174, Reg: 0.000000
	Iter [30/68] Loss: 0.2887, Reg: 0.000000
	Iter [40/68] Loss: 0.3907, Reg: 0.000000
	Iter [50/68] Loss: 0.3371, Reg: 0.000000
	Iter [60/68] Loss: 0.3725, Reg: 0.000000
Training loss = 0.3368
Time taken: 136.31 sec.

Evaluation:
	Val Acc: 74.00 - Loss: 0.6843
	Current best val acc: 77.00

-----------------------------------
Epoch: [232/10000]
	Iter [10/68] Loss: 0.1828, Reg: 0.000000
	Iter [20/68] Loss: 0.3115, Reg: 0.000000
	Iter [30/68] Loss: 0.4107, Reg: 0.000000
	Iter [40/68] Loss: 0.2534, Reg: 0.000000
	Iter [50/68] Loss: 0.4067, Reg: 0.000000
	Iter [60/68] Loss

KeyboardInterrupt: 

In [ ]:
#Check input pipeline throughput
start = time.time()
for i, (inputs, targets) in enumerate(train_loader):
    #inputs = augmentor.augment_on_GPU(inputs)
    if i==0:
        print("Input tensor shape: ",inputs.shape)
    
    sys.stdout.write('Reading batch {} of {} \r'.format(i+1,len(train_loader)))
    sys.stdout.flush()
#    if i==10:
#        break
print('\nTime to read one epoch:  %.2f seconds.' % (time.time() - start))

In [ ]:
_,model = next(resnet.named_children())
for name,module in model.named_children():
    if name in ['fc']:
        print(module.weight)

In [ ]:
print(inputs[0:2,:].shape)

A = augment_on_GPU(inputs)
A = torch.from_numpy(A)
print(A.shape)

In [ ]:
print(inputs[0,0,:].shape)

C = A-inputs
B = C[0,5,:].transpose(0, 2)
B = B.transpose(0,1)
plt.imshow(B)


In [ ]:
!jupyter nbconvert --to script interactive_controller.ipynb --output controller

In [ ]:
torch.cuda.empty_cache()


In [ ]:
import subprocess
A = str(subprocess.check_output(["{job_id}"]))
print("s = ",A)
#print(subprocess.call("nvidia-smi"))

In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd

output = np.concatenate(test_output,axis=0)
print(output.shape)
target = np.concatenate(test_target,axis=0)
print(target.shape)
Decision = np.argmax(output,axis=1)
print(Decision.shape)
df = pd.DataFrame(output,columns=['Standard', 'Acc_B', 'Abs_RB7'])
df['Decision'] = Decision[:,np.newaxis]
df['Target']   = target[:,np.newaxis]
print(df.head(2))
df.to_csv('/Shared/CTmechanics_COPDGene/Amin/Report/COPDGene/TestOutput.csv',index=False)


In [ ]:
!hostname